<a href="https://colab.research.google.com/github/mryab/efficient-dl-systems/blob/main/week04_large_models/practice_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Efficient DL Practice: Advanced Parallelism (5 points)

In this practice session, we'll cover techniques for training large models in parallel: **Model** and **Sequence Parallelism**.
More precisely, you will implement them, and we will root for you as you go. Good luck, 🥩👜!



In [1]:
# dependencies: the code will likely work with slightly newer/older versions, but may require minimal patching
# %pip install -q transformers==4.48.3 peft==0.14.0

import transformers; assert transformers.__version__.startswith("4.48"), transformers.__version__
import peft; assert peft.__version__.startswith("0.14"), peft.__version__

__Part 1: Tensor Parallelism (2 points)__
![img](https://pytorch.org/tutorials/_images/megatron_lm.png)

We'll begin by implementing a simple tensor parallelism (also known as the [original](https://papers.nips.cc/paper_files/paper/2012/hash/c399862d3b9d6b76c8436e924a68c45b-Abstract.html) model parallelism).

Our ultimate objective is to run and fine-tune a Llama 3.x model in tensor-parallel mode. However, it is rather difficult to do that in one go, especially if you take bugs into account. So we'll start simple: __here's a single Llama MLP module:__

`please read the code below carefully, it's a template for the remaining assgnments`.

In [2]:
%%writefile tensor_parallel_mlp.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist


class LlamaMLP(nn.Module):  #  based on llama 3.1 8B configuration
    def __init__(self, hidden_size: int = 4096, intermediate_size: int = 14336):
        super().__init__()
        self.gate_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.up_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.down_proj = nn.Linear(intermediate_size, hidden_size, bias=False)

    def forward(self, input):
        return self.down_proj(F.silu(self.gate_proj(input)) * self.up_proj(input))


class ComputeWithAllReduce(torch.autograd.Function):
    @staticmethod  # fun fact: torch.distributed.nn has differentiable all_reduce!
    def forward(ctx, tp_shard: nn.Module, input: torch.Tensor):
        input = input.detach().requires_grad_(input.requires_grad)
        ctx.save_for_backward(input)
        ctx._tp_shard = tp_shard
        output = tp_shard(input)
        dist.all_reduce(output)
        return output
    @staticmethod
    def backward(ctx, grad_output: torch.Tensor):
        with torch.enable_grad():
          output = ctx._tp_shard(ctx.saved_tensors[0])
          output.backward(grad_output)
        dist.all_reduce(ctx.saved_tensors[0].grad)
        return None, ctx.saved_tensors[0].grad


class AllReduceModule(nn.Sequential):
    def forward(self, input: torch.Tensor):
        return ComputeWithAllReduce.apply(super().forward, input)


if __name__ == "__main__":
    dist.init_process_group("gloo")   # use nccl for cuda devices
    torch.manual_seed(1337)           # init weights equally on all ranks
    rank, world_size = dist.get_rank(), dist.get_world_size()

    for active_rank in range(world_size):
      dist.barrier()  # initialize each rank sequentially to save system RAM
      if rank != active_rank: continue

      # we will now implement Tensor Parallelism for the ref_module below:
      ref_module = nn.Sequential(nn.RMSNorm(4096), LlamaMLP())
      # compute reference tensors to test against them later
      input = torch.randn(1, 4096, requires_grad=True)
      ref_output = ref_module(input)
      ref_output.sum().backward()
      ref_input_grad = input.grad.clone()

      # TP step 1: define a module that computes a portion of intermediate units
      intermediate_size = ref_module[1].down_proj.in_features
      local_units = intermediate_size // world_size
      assert intermediate_size % world_size == 0
      tp_module = nn.Sequential(   # assign a portion of units per rank --v
          nn.RMSNorm(4096), AllReduceModule(LlamaMLP(intermediate_size=local_units))
      )   # all-reduce outputs during forward, all-reduce gradients on backward

      with torch.no_grad():  # copy select weights from the reference MLP
        # v-- input norm layer is too small to bother parallelizing - we replicate it!
        tp_module[0].load_state_dict(ref_module[0].state_dict())
        # up and gate projections are sharded across output units
        unit_slice = slice(local_units * rank, local_units * (rank + 1))
        tp_module[1][0].up_proj.weight[...] = ref_module[1].up_proj.weight[unit_slice]
        tp_module[1][0].gate_proj.weight[...] = ref_module[1].gate_proj.weight[unit_slice]
        # down projection is sharded across input units, matching up/gate proj
        tp_module[1][0].down_proj.weight[...] = ref_module[1].down_proj.weight[:, unit_slice]
      print(f"Initialized {rank=}", flush=True)
      del ref_module  # free RAM for next rank

    dist.barrier()  # test 1: forward pass
    tp_input = input.detach().requires_grad_(True)
    tp_output = tp_module(tp_input)
    if rank == 0:
        print(f"\nReference outputs ({rank=}):", ref_output.data, flush=True)
    for i in range(world_size):
        dist.barrier()
        if i != rank: continue
        print(f"TParallel outputs ({rank=}):", tp_output.data, flush=True)
        assert torch.allclose(tp_output, ref_output, atol=1e-4), f"output mismatch on {rank=}"

    dist.barrier()  # test 2: backward w.r.t. inputs
    assert tp_input.grad is None
    tp_output.sum().backward()
    if rank == 0:
        print(f"\nReference input grad ({rank=}):", ref_input_grad, flush=True)
    for i in range(world_size):
        dist.barrier()
        if i != rank: continue
        print(f"TParallel input grad ({rank=}):", tp_input.grad.data, flush=True)
        assert torch.allclose(tp_input.grad, ref_input_grad, atol=1e-4), f"input_grad mismatch on {rank=}"


Overwriting tensor_parallel_mlp.py


In [3]:
!OMP_NUM_THREADS=1 torchrun --nproc_per_node 2 tensor_parallel_mlp.py

[Gloo] Rank 1 is connected to 1 peer ranks. Expected number of connected peer ranks is : 1
[Gloo] Rank 0 is connected to 1 peer ranks. Expected number of connected peer ranks is : 1
Initialized rank=0
Initialized rank=1

Reference outputs (rank=0): tensor([[-0.1145,  0.0160,  0.0500,  ..., -0.1455,  0.1126, -0.0192]])
TParallel outputs (rank=0): tensor([[-0.1145,  0.0160,  0.0500,  ..., -0.1455,  0.1126, -0.0192]])
TParallel outputs (rank=1): tensor([[-0.1145,  0.0160,  0.0500,  ..., -0.1455,  0.1126, -0.0192]])

Reference input grad (rank=0): tensor([[ 0.0343, -0.2492, -0.1858,  ..., -0.0541,  0.0388, -0.1529]])
TParallel input grad (rank=0): tensor([[ 0.0343, -0.2492, -0.1858,  ..., -0.0541,  0.0388, -0.1529]])
TParallel input grad (rank=1): tensor([[ 0.0343, -0.2492, -0.1858,  ..., -0.0541,  0.0388, -0.1529]])


Note that the code above lacks two details:
- it uses a form of checkpointing, but does not save random state, which would be required if you use dropout;
- it replicates RMSNorm, but it is not synchronized. Training would require all-reduce-ing gradients for those layers, e.g. by wrapping them with DDP.

```

```

```

```

```

```


__Task 1 (1 point):__ Implement tensor-parallel multi-head attention.

Like with the MLP module before, you can partition attention across multiple devices. This time, every device is to compute a portion of whole attention **heads** (and not individual units). We exploit the property that an multi-head attention layer can be viewed as a sum of individual head outputs after output projection.

For the sake of formality, this is the computation you need to parallelize:

In [4]:
import torch
from transformers.models.llama.modeling_llama import LlamaConfig, LlamaAttention, LlamaRotaryEmbedding
MODEL_NAME = "unsloth/Llama-3.2-1B"  # for testing (but not grading!), you may want to use Maykeye/TinyLLama-v0
config = LlamaConfig.from_pretrained(MODEL_NAME)
layer = LlamaAttention(config, layer_idx=5)
rotary_emb = LlamaRotaryEmbedding(config)

input = torch.randn(1, 128, config.hidden_size, requires_grad=True)
position_embeddings = rotary_emb(input, position_ids=torch.arange(128)[None])

output, *_etc = layer(input, attention_mask=None, position_embeddings=position_embeddings)
print(f"{output=}")
output.norm().backward()
print(f"{input.grad=}")

output=tensor([[[ 0.0154,  0.0549,  0.0559,  ...,  0.0129,  0.0206,  0.0052],
         [ 0.0193,  0.0483,  0.0460,  ...,  0.0106,  0.0001, -0.0131],
         [ 0.0106,  0.0493,  0.0357,  ...,  0.0233, -0.0114,  0.0006],
         ...,
         [ 0.0092,  0.0660,  0.0353,  ...,  0.0057, -0.0149,  0.0451],
         [ 0.0254,  0.0371,  0.0443,  ...,  0.0201,  0.0063,  0.0100],
         [ 0.0117,  0.0552,  0.0449,  ...,  0.0130, -0.0110, -0.0079]]],
       grad_fn=<UnsafeViewBackward0>)
input.grad=tensor([[[-0.0015, -0.0002,  0.0002,  ...,  0.0024, -0.0006,  0.0011],
         [-0.0016, -0.0003,  0.0003,  ...,  0.0025, -0.0007,  0.0009],
         [-0.0016, -0.0003,  0.0003,  ...,  0.0026, -0.0007,  0.0010],
         ...,
         [-0.0016, -0.0003,  0.0002,  ...,  0.0024, -0.0007,  0.0009],
         [-0.0016, -0.0002,  0.0004,  ...,  0.0026, -0.0007,  0.0009],
         [-0.0016, -0.0002,  0.0002,  ...,  0.0025, -0.0006,  0.0011]]])


Same as before, your task is to create a multi-head attention layer, partition it across ranks and verify two things:
- attention outputs on the same inputs (and mask) match with the non-parallel version;
- gradients w.r.t. attention inputs are the same; gradients w.r.t. mask need not be verified.


In [5]:
config

LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.48.3",
  "unsloth_fixed": true,
  "use_cache": true,
  "vocab_size": 128256
}

In [26]:
%%writefile tensor_parallel_attn.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist

from transformers.models.llama.modeling_llama import LlamaConfig, LlamaAttention, LlamaRotaryEmbedding
from torch import nn
import torch
from typing import Dict, Callable, Optional, Tuple
from copy import deepcopy


MODEL_NAME = "unsloth/Llama-3.2-1B"  # for testing (but not grading!), you may want to use Maykeye/TinyLLama-v0


def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    This is the equivalent of torch.repeat_interleave(x, dim=1, repeats=n_rep). The hidden states go from (batch,
    num_key_value_heads, seqlen, head_dim) to (batch, num_attention_heads, seqlen, head_dim)
    """
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)


def sdpa_attention_forward(
    module: torch.nn.Module,
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    attention_mask: Optional[torch.Tensor],
    dropout: float = 0.0,
    scaling: Optional[float] = None,
    is_causal: Optional[bool] = None,
    **kwargs,
) -> Tuple[torch.Tensor, None]:
    if hasattr(module, "num_key_value_groups"):
        key = repeat_kv(key, module.num_key_value_groups)
        value = repeat_kv(value, module.num_key_value_groups)

    causal_mask = attention_mask
    if attention_mask is not None:
        causal_mask = causal_mask[:, :, :, : key.shape[-2]]

    query = query.contiguous()
    key = key.contiguous()
    value = value.contiguous()

    if is_causal is None:
        is_causal = causal_mask is None and query.shape[2] > 1

    if torch.jit.is_tracing() and isinstance(is_causal, torch.Tensor):
        is_causal = is_causal.item()

    # print('query', query.shape)
    # print('key', key.shape)
    # print('value', value.shape)
    
    attn_output = torch.nn.functional.scaled_dot_product_attention(
        query,
        key,
        value,
        attn_mask=causal_mask,
        dropout_p=dropout,
        scale=scaling,
        is_causal=is_causal,
    )
    attn_output = attn_output.transpose(1, 2).contiguous()

    return attn_output, None


def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(q, k, cos, sin, position_ids=None, unsqueeze_dim=1):
    cos = cos.unsqueeze(unsqueeze_dim)
    sin = sin.unsqueeze(unsqueeze_dim)
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed


class MyLlamaAttention(nn.Module):
    """Multi-headed attention from 'Attention Is All You Need' paper"""

    def __init__(self, config: LlamaConfig, layer_idx: int):
        super().__init__()
        self.config = config
        self.layer_idx = layer_idx
        self.head_dim = getattr(config, "head_dim", config.hidden_size // config.num_attention_heads)
        self.num_key_value_groups = config.num_attention_heads // config.num_key_value_heads
        self.scaling = self.head_dim**-0.5
        self.attention_dropout = config.attention_dropout
        self.is_causal = True

        self.q_proj = nn.Linear(
            config.hidden_size, config.num_attention_heads * self.head_dim, bias=config.attention_bias
        )
        self.k_proj = nn.Linear(
            config.hidden_size, config.num_key_value_heads * self.head_dim, bias=config.attention_bias
        )
        self.v_proj = nn.Linear(
            config.hidden_size, config.num_key_value_heads * self.head_dim, bias=config.attention_bias
        )
        self.o_proj = nn.Linear(
            config.num_attention_heads * self.head_dim, config.hidden_size, bias=config.attention_bias
        )

    def forward(
        self,
        hidden_states: torch.Tensor,
        position_embeddings: Tuple[torch.Tensor, torch.Tensor],
        attention_mask: Optional[torch.Tensor],
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        input_shape = hidden_states.shape[:-1]
        hidden_shape = (*input_shape, -1, self.head_dim)

        query_states = self.q_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        key_states = self.k_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        value_states = self.v_proj(hidden_states).view(hidden_shape).transpose(1, 2)

        cos, sin = position_embeddings
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)

        # print('query_states', query_states.shape)
        # print('key_states', key_states.shape)
        # print('value_states', value_states.shape)
        
        attn_output, attn_weights = sdpa_attention_forward(
            self,
            query_states,
            key_states,
            value_states,
            attention_mask,
            dropout=0.0 if not self.training else self.attention_dropout,
            scaling=self.scaling,
        )

        # print('attn_output', attn_output.shape)

        attn_output = attn_output.reshape(*input_shape, -1).contiguous()
        attn_output = self.o_proj(attn_output)
        return attn_output


class ComputeWithAllReduce(torch.autograd.Function):
    @staticmethod  # fun fact: torch.distributed.nn has differentiable all_reduce!
    def forward(ctx, tp_shard: nn.Module, input: torch.Tensor, **kwargs):
        input = input.detach().requires_grad_(input.requires_grad)
        ctx.save_for_backward(input)
        ctx._tp_shard = tp_shard
        output = tp_shard(input, **kwargs)
        dist.all_reduce(output)
        return output
    @staticmethod
    def backward(ctx, grad_output: torch.Tensor):
        with torch.enable_grad():
          output = ctx._tp_shard(ctx.saved_tensors[0])
          output.backward(grad_output)
        dist.all_reduce(ctx.saved_tensors[0].grad)
        return None, ctx.saved_tensors[0].grad


# class AllReduceModule(nn.Sequential):
#     def forward(self, input: torch.Tensor, **kwargs):
#         fn = lambda x: super(AllReduceModule, self).forward(x, **kwargs)
#         return ComputeWithAllReduce.apply(fn, input)

class ReduceWrapper(nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module

    def forward(self, input, **kwargs):
        fn = lambda x: self.module(x, **kwargs)
        return ComputeWithAllReduce.apply(fn, input)



if __name__ == "__main__":
    dist.init_process_group("nccl")   # use nccl for cuda devices
    torch.manual_seed(1337)           # init weights equally on all ranks
    rank, world_size = dist.get_rank(), dist.get_world_size()
    torch.cuda.set_device(rank)

    for active_rank in range(world_size):
      dist.barrier()  # initialize each rank sequentially to save system RAM
      if rank != active_rank: continue

      # we will now implement Tensor Parallelism for the ref_module below:
      config = LlamaConfig.from_pretrained(MODEL_NAME)
      ref_module = MyLlamaAttention(config, layer_idx=5).cuda()
      rotary_emb = LlamaRotaryEmbedding(config).cuda()
      # ^-- you may need to modify this code, e.g. pass parameters or use transformers LlamaAttention (as above)

      # generate reference tensors to test against them later
      input = torch.randn(1, 128, 2048, requires_grad=True, device='cuda')
      position_embeddings = rotary_emb(input, position_ids=torch.arange(128)[None].cuda())
      extra_inputs = dict(attention_mask=None, position_embeddings=position_embeddings)  # <-- OPTIONAL: either design additional inputs here, as in the reference above

      ref_output = ref_module(input, **extra_inputs).cuda()
      ref_output.sum().backward()
      ref_input_grad = input.grad.clone()

      # TP step 1: define a module that computes a portion of attention heads

      smaller_config = deepcopy(config)
      assert config.num_attention_heads % world_size == 0
      smaller_config.num_attention_heads = config.num_attention_heads // world_size
      smaller_config.num_key_value_heads = config.num_key_value_heads // world_size
      tp_module = ReduceWrapper(MyLlamaAttention(smaller_config, layer_idx=5)).cuda()  # create a tensor-parallel version of the Attention module

      with torch.no_grad():
        #   print('hidden_size', config.hidden_size)
        #   print('num_key_value_heads', smaller_config.num_key_value_heads)
        #   print('num_attention_heads', smaller_config.num_attention_heads)
        #   print('orig num_key_value_heads', config.num_key_value_heads)
        #   print('orig num_attention_heads', config.num_attention_heads)
        #   print('ref_module.k_proj.weight.data', ref_module.k_proj.weight.data.shape)
          local_units = config.head_dim * smaller_config.num_key_value_heads
          unit_slice = slice(local_units * rank, local_units * (rank + 1))
          tp_module.module.k_proj.weight.data = ref_module.k_proj.weight.data[unit_slice]
          tp_module.module.v_proj.weight.data = ref_module.v_proj.weight.data[unit_slice]
          local_units = config.head_dim * smaller_config.num_attention_heads
          unit_slice = slice(local_units * rank, local_units * (rank + 1))
        #   print('tp_module[0].o_proj.weight.data', tp_module[0].o_proj.weight.data.shape)
        #   print('ref_module.o_proj.weight.data', ref_module.o_proj.weight.data.shape)
          tp_module.module.o_proj.weight.data[...] = ref_module.o_proj.weight.data[:, unit_slice]
          tp_module.module.q_proj.weight.data[...] = ref_module.q_proj.weight.data[unit_slice]

      print(f"Initialized {rank=}", flush=True)
      del ref_module  # free RAM for next rank

    # TEST AREA: you are free to add additional parameters, but your code *must* run the same tests as below
    dist.barrier()  # test 1: forward pass
    tp_input = input.detach().cuda().requires_grad_(True)
    tp_output = tp_module(tp_input, **extra_inputs).cuda()
    print(ref_output.data)
    if rank == 0:
        print(f"\nReference outputs ({rank=}):", ref_output.data, flush=True)
    for i in range(world_size):
        dist.barrier()
        if i != rank: continue
        print(f"TParallel outputs ({rank=}):", tp_output.data, flush=True)
        assert torch.allclose(tp_output, ref_output, atol=1e-5), f"output mismatch on {rank=}"

    dist.barrier()  # test 2: backward w.r.t. inputs
    assert tp_input.grad is None
    tp_output.sum().backward()
    if rank == 0:
        print(f"\nReference input grad ({rank=}):", ref_input_grad, flush=True)
    for i in range(world_size):
        dist.barrier()
        if i != rank: continue
        print(f"TParallel input grad ({rank=}):", tp_input.grad.data, flush=True)
        assert torch.allclose(tp_input.grad, ref_input_grad, atol=1e-4), f"input_grad mismatch on {rank=}"


Overwriting tensor_parallel_attn.py


In [27]:
!OMP_NUM_THREADS=1 torchrun --nproc_per_node 2 tensor_parallel_attn.py
# ^-- feel free to modify parameters, as long as there are at least 2 ranks

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/venv/main/lib/python3.10/site-packages/torch/distributed/c10d_logger.py:83: UserWarning: barrier(): using the device under current context. You can specify `device_id` in `init_process_group` to mute this warning.
  return func(*args, **kwargs)
[rank0]:[W223 08:15:40.920344842 ProcessGroupNCCL.cpp:5138] Guessing device ID based on global rank. This can cause a hang if rank to GPU mapping is heterogeneous. You can specify device_id in init_process_group()
Initialized rank=0
Initialized rank=1
tensor([[[ 2.2773e-01,  7.7059e-01, -1.7579e-02,  ...,  3.0142e-02,
           2.6831e-01,  2.4405e-01],
         [ 1.7466e-02,  5.4525e-03, -1.2642e-01,  ..., -4.0684e-02,
           5.9789e-02, -3.3783e-01],
         [-2.4119e-03,  1.2824e-01,  8.7102e-02,  ...,  1.6749e-01,
           4.9222e-02, -1.8924e-02],
         ...,
         [ 4.8462e-02,  6.5924e-02,  1.1517e-03,  ...,  1.2480e-02,
           2.9387e-02,  5.1865e-03],
         [ 5.5688e-02,  5.9044e-02,  2.2112e-02,  ..., -1.8325e-04,


Well done! *(hopefully. If not, go back and, well... do it)*

```

```


```

```


```

```


```

```


```

```


### Full model conversion

Now let's apply this technique to parallelize the actual Llama model. As in, with weights.

__Task 2 (1 point):__ Combine the two previous techniques in one file that parallelizes an actual Llama model and .generates meaningful output. For simplicity, you do not need to partition key-value cache here - only the forward pass itself. We will default to generating tokens with recomputation.

For the sake of formality, your task is to parallelize the following inference code:


In [12]:
import torch
import transformers
MODEL_NAME = "unsloth/Llama-3.2-1B"  # for testing (but not grading!), you may want to use Maykeye/TinyLLama-v0

tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
model = transformers.LlamaForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float32)  # <-- you are allowed to switch to bf16

prompt = "A quick brown fox"
input_ids = tokenizer(prompt, return_tensors='pt')["input_ids"]
print(end=prompt)
for i in range(5):
  with torch.no_grad():
    new_token = model(input_ids).logits[0, -1].argmax(-1)
    input_ids = torch.cat([input_ids, new_token.view(1, 1)], dim=1)
  print(end=tokenizer.decode(new_token), flush=True)
# pro tip: delete the model or restart session to free RAM for the TP experiments

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

A quick brown fox jumps over the lazy dog


**Requirements:** your code must do the following things for the full grade:
- instantiate an actually trained Llama model (Llama 3.2 1B or larger is fine, maykeye is not)
- run forward pass with at least 2 ranks and verify that the logits are close,
- run backward pass w.r.t. non-parallelized input embeddings, verify that the gradients are close,
- perform inference for 10 steps to verify that the model produces meaningful outputs (see below)

You are only required to tensor-parallel-ize the transformer layers. Parallelizing embeddings and logits is optional. If you do choose to parallelize embeddings, we sincerely recommend that you partition across the embedding dim, not across tokens - so that the computation is balanced.

In [13]:
%%writefile tensor_parallel_llama.py
import torch
import transformers
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist

from transformers.models.llama.modeling_llama import LlamaConfig, LlamaAttention, LlamaRotaryEmbedding
from torch import nn
import torch
from typing import Dict, Callable, Optional, Tuple
from copy import deepcopy


MODEL_NAME = "unsloth/Llama-3.2-1B"  # for testing (but not grading!), you may want to use Maykeye/TinyLLama-v0


def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    This is the equivalent of torch.repeat_interleave(x, dim=1, repeats=n_rep). The hidden states go from (batch,
    num_key_value_heads, seqlen, head_dim) to (batch, num_attention_heads, seqlen, head_dim)
    """
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)


def sdpa_attention_forward(
    module: torch.nn.Module,
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    attention_mask: Optional[torch.Tensor],
    dropout: float = 0.0,
    scaling: Optional[float] = None,
    is_causal: Optional[bool] = None,
    **kwargs,
) -> Tuple[torch.Tensor, None]:
    if hasattr(module, "num_key_value_groups"):
        key = repeat_kv(key, module.num_key_value_groups)
        value = repeat_kv(value, module.num_key_value_groups)

    causal_mask = attention_mask
    if attention_mask is not None:
        causal_mask = causal_mask[:, :, :, : key.shape[-2]]

    query = query.contiguous()
    key = key.contiguous()
    value = value.contiguous()

    if is_causal is None:
        is_causal = causal_mask is None and query.shape[2] > 1

    if torch.jit.is_tracing() and isinstance(is_causal, torch.Tensor):
        is_causal = is_causal.item()

    # print('query', query.shape)
    # print('key', key.shape)
    # print('value', value.shape)
    
    attn_output = torch.nn.functional.scaled_dot_product_attention(
        query,
        key,
        value,
        attn_mask=causal_mask,
        dropout_p=dropout,
        scale=scaling,
        is_causal=is_causal,
    )
    attn_output = attn_output.transpose(1, 2).contiguous()

    return attn_output, None


def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(q, k, cos, sin, position_ids=None, unsqueeze_dim=1):
    cos = cos.unsqueeze(unsqueeze_dim)
    sin = sin.unsqueeze(unsqueeze_dim)
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed


class MyLlamaAttention(nn.Module):
    """Multi-headed attention from 'Attention Is All You Need' paper"""

    def __init__(self, config: LlamaConfig, layer_idx: int):
        super().__init__()
        self.config = config
        self.layer_idx = layer_idx
        self.head_dim = getattr(config, "head_dim", config.hidden_size // config.num_attention_heads)
        self.num_key_value_groups = config.num_attention_heads // config.num_key_value_heads
        self.scaling = self.head_dim**-0.5
        self.attention_dropout = config.attention_dropout
        self.is_causal = True

        self.q_proj = nn.Linear(
            config.hidden_size, config.num_attention_heads * self.head_dim, bias=config.attention_bias
        )
        self.k_proj = nn.Linear(
            config.hidden_size, config.num_key_value_heads * self.head_dim, bias=config.attention_bias
        )
        self.v_proj = nn.Linear(
            config.hidden_size, config.num_key_value_heads * self.head_dim, bias=config.attention_bias
        )
        self.o_proj = nn.Linear(
            config.num_attention_heads * self.head_dim, config.hidden_size, bias=config.attention_bias
        )

    def forward(
        self,
        hidden_states: torch.Tensor,
        position_embeddings: Tuple[torch.Tensor, torch.Tensor],
        attention_mask: Optional[torch.Tensor], **kwargs,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        input_shape = hidden_states.shape[:-1]
        hidden_shape = (*input_shape, -1, self.head_dim)

        query_states = self.q_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        key_states = self.k_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        value_states = self.v_proj(hidden_states).view(hidden_shape).transpose(1, 2)

        cos, sin = position_embeddings
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)

        # print('query_states', query_states.shape)
        # print('key_states', key_states.shape)
        # print('value_states', value_states.shape)
        
        attn_output, attn_weights = sdpa_attention_forward(
            self,
            query_states,
            key_states,
            value_states,
            attention_mask,
            dropout=0.0 if not self.training else self.attention_dropout,
            scaling=self.scaling,
        )

        # print('attn_output', attn_output.shape)

        attn_output = attn_output.reshape(*input_shape, -1).contiguous()
        attn_output = self.o_proj(attn_output)
        return attn_output


class ComputeWithAllReduce(torch.autograd.Function):
    @staticmethod  # fun fact: torch.distributed.nn has differentiable all_reduce!
    def forward(ctx, tp_shard: nn.Module, input: torch.Tensor, **kwargs):
        input = input.detach().requires_grad_(input.requires_grad)
        ctx.save_for_backward(input)
        ctx._tp_shard = tp_shard
        output = tp_shard(input, **kwargs)
        dist.all_reduce(output)
        return output
    @staticmethod
    def backward(ctx, grad_output: torch.Tensor):
        with torch.enable_grad():
          output = ctx._tp_shard(ctx.saved_tensors[0])
          output.backward(grad_output)
        dist.all_reduce(ctx.saved_tensors[0].grad)
        return None, ctx.saved_tensors[0].grad


class ReduceWrapper(nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module

    def forward(self, hidden_states=None, **kwargs):
        fn = lambda x: self.module(x, **kwargs)
        output = ComputeWithAllReduce.apply(fn, hidden_states)
        return output, None


class LlamaMLP(nn.Module):  #  based on llama 3.1 8B configuration
    def __init__(self, hidden_size: int = 4096, intermediate_size: int = 14336):
        super().__init__()
        self.gate_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.up_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.down_proj = nn.Linear(intermediate_size, hidden_size, bias=False)

    def forward(self, input):
        return self.down_proj(F.silu(self.gate_proj(input)) * self.up_proj(input))


class ComputeWithAllReduce(torch.autograd.Function):
    @staticmethod  # fun fact: torch.distributed.nn has differentiable all_reduce!
    def forward(ctx, tp_shard: nn.Module, input: torch.Tensor):
        input = input.detach().requires_grad_(input.requires_grad)
        ctx.save_for_backward(input)
        ctx._tp_shard = tp_shard
        output = tp_shard(input)
        dist.all_reduce(output)
        return output
    @staticmethod
    def backward(ctx, grad_output: torch.Tensor):
        with torch.enable_grad():
          output = ctx._tp_shard(ctx.saved_tensors[0])
          output.backward(grad_output)
        dist.all_reduce(ctx.saved_tensors[0].grad)
        return None, ctx.saved_tensors[0].grad


class AllReduceModule(nn.Sequential):
    def forward(self, input: torch.Tensor):
        return ComputeWithAllReduce.apply(super().forward, input)


if __name__ == "__main__":
    dist.init_process_group("gloo")   # use nccl for cuda devices
    torch.manual_seed(1337)           # init weights equally on all ranks
    rank, world_size = dist.get_rank(), dist.get_world_size()

    for active_rank in range(world_size):
        dist.barrier()
        if rank != active_rank: continue
        
        MODEL_NAME = "unsloth/Llama-3.2-1B"  # for testing (but not grading!), you may want to use Maykeye/TinyLLama-v0
        config = LlamaConfig.from_pretrained(MODEL_NAME)
        tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
        ref_model = transformers.LlamaForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float32)  # <-- you are allowed to switch to bf16
        prompt = "A quick brown fox"
        input_ids = tokenizer(prompt, return_tensors='pt')["input_ids"]
        input = ref_model.model.embed_tokens(input_ids).detach().requires_grad_(True)
        ref_output = ref_model(inputs_embeds=input).logits
        ref_output.mean().backward()
        ref_input_grad = input.grad.clone()
        
        tp_model = deepcopy(ref_model)
        
        with torch.no_grad():
            for i in range(len(tp_model.model.layers)):
                smaller_config = deepcopy(config)
                smaller_config.num_attention_heads = config.num_attention_heads // world_size
                smaller_config.num_key_value_heads = config.num_key_value_heads // world_size
                layer = ReduceWrapper(MyLlamaAttention(smaller_config, layer_idx=i))
                local_units = config.head_dim * smaller_config.num_key_value_heads
                unit_slice = slice(local_units * rank, local_units * (rank + 1))
                layer.module.k_proj.weight.data = ref_model.model.layers[i].self_attn.k_proj.weight.data[unit_slice]
                layer.module.v_proj.weight.data = ref_model.model.layers[i].self_attn.v_proj.weight.data[unit_slice]
                local_units = config.head_dim * smaller_config.num_attention_heads
                unit_slice = slice(local_units * rank, local_units * (rank + 1))
                layer.module.o_proj.weight.data[...] = ref_model.model.layers[i].self_attn.o_proj.weight.data[:, unit_slice]
                layer.module.q_proj.weight.data[...] = ref_model.model.layers[i].self_attn.q_proj.weight.data[unit_slice]
                tp_model.model.layers[i].self_attn = layer

                intermediate_size = ref_model.model.layers[i].mlp.down_proj.in_features
                local_units = intermediate_size // world_size
                
                mlp_layer = AllReduceModule(LlamaMLP(hidden_size=config.hidden_size, intermediate_size=local_units))
                unit_slice = slice(local_units * rank, local_units * (rank + 1))
                
                mlp_layer[0].down_proj.weight.data[...] = ref_model.model.layers[i].mlp.down_proj.weight.data[:, unit_slice]
                mlp_layer[0].up_proj.weight.data[...] = ref_model.model.layers[i].mlp.up_proj.weight.data[unit_slice]
                mlp_layer[0].gate_proj.weight.data[...] = ref_model.model.layers[i].mlp.gate_proj.weight.data[unit_slice]
                tp_model.model.layers[i].mlp = mlp_layer
    
    dist.barrier()
    tp_input = input.detach().requires_grad_(True)
    tp_output = tp_model(inputs_embeds=tp_input).logits
    if rank == 0:
        print(f"\nReference outputs ({rank=}):", ref_output.data, flush=True)
    for i in range(world_size):
        dist.barrier()
        if i != rank: continue
        print(f"TParallel outputs ({rank=}):", tp_output.data, flush=True)
        assert torch.allclose(tp_output, ref_output, atol=1e-4), f"output mismatch on {rank=}"
    
    dist.barrier()  # test 2: backward w.r.t. inputs
    assert tp_input.grad is None
    tp_output.mean().backward()
    if rank == 0:
        print(f"\nReference input grad ({rank=}):", ref_input_grad, flush=True)
    for i in range(world_size):
        dist.barrier()
        if i != rank: continue
        print(f"TParallel input grad ({rank=}):", tp_input.grad.data, flush=True)
        assert torch.allclose(tp_input.grad, ref_input_grad, atol=1e-4), f"input_grad mismatch on {rank=}"
    
    dist.barrier()
    text = "A quick brown"
    input_ids = tokenizer(text, return_tensors='pt')["input_ids"]
    for i in range(10):
        with torch.no_grad():
            new_token = tp_model(input_ids).logits[0, -1].argmax(-1)
            input_ids = torch.cat([input_ids, new_token.view(1, 1)], dim=1)
        # print(end=tokenizer.decode(new_token), flush=True)
    final_text = tokenizer.decode(input_ids[0])
    print(end=f"\n\nText generation ({rank=}):\n{final_text}\n", flush=True)
    dist.barrier()

Writing tensor_parallel_llama.py


In [14]:
!OMP_NUM_THREADS=1 torchrun --nproc_per_node 2 tensor_parallel_llama.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[Gloo] Rank 1 is connected to 1 peer ranks. Expected number of connected peer ranks is : 1
[Gloo] Rank 0 is connected to 1 peer ranks. Expected number of connected peer ranks is : 1

Reference outputs (rank=0): tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [ 8.7207,  7.6674,  7.7454,  ..., -1.3887, -1.3885, -1.3885],
         [ 9.7538,  7.3480,  5.6021,  ..., -0.3843, -0.3842, -0.3834],
         [10.0018,  7.8891,  5.5960,  ..., -2.3212, -2.3206, -2.3199],
         [12.8186, 10.7330,  8.5906,  ..., -1.0288, -1.0301, -1.0296]]])
TParallel outputs (rank=0): tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [ 8.7207,  7.6674,  7.7454,  ..., -1.3887, -1.3885, -1.3885],
         [ 9.7538,  7.3480,  5.6021,  ..., -0.3843, -0.3842, -0.3834],
         [10.0018,  7.8891,  5.5960,  ..., -2.3212, -2.3206, -2.3199],
         [12.8186, 10.7330,  8.5906,  ..., -1.0288, -1.0301, -1.0296]]])
TParallel outputs (rank=1): tensor([[[ 7.0544, 

```

```

```

```

```

```

```

```

```

```

```

```

### Using [`torch.distributed.tensor`](https://pytorch.org/docs/stable/distributed.tensor.html)

PyTorch has an in-built functionality called [DTensor](https://pytorch.org/docs/stable/distributed.tensor.html), designed to help implementing tensor-level parallelism with various sharding strategies. This includes Tensor parallelism itself, as well as other techniques such as Sequence Parallelism, as they are both, essentially, parallelism across different tensor dimensions.

__Task 3 (1 point):__ Your next task will be to replicate your previous code (llama inference) using DTensor instead of manual AllReduce. We recommend you start by skimming the [documentation for DTensor](https://pytorch.org/docs/stable/distributed.tensor.html) to learn the interface and [the minimal example](https://github.com/pytorch/examples/blob/main/distributed/tensor_parallelism/tensor_parallel_example.py) to learn how to put the pieces together.


We recommend that you dedicate some time to learn and play with it before you proceed to parallelize Llama.

The main objective is the same as in the previous task - run .generate with DTensor - and then compare it against the manual implementation. **Please report at least some speed comparison for forward and backward passes between this and the previous task.** If absolutely impossible (e.g. you don't have multiple gpus), we can accept a fallback assignment of implementing basic training: overfit the model to a single batch (like task 5 below) and demonstrate that it works - if you choose this option, say so in bold, large-font letters somewhere where the grader can see.

But first, here's a quick demo of using DTensor for simple matrix multiplication - meant as a testbed for your experiments.

In [15]:
from torch.distributed._tensor import DTensor

In [16]:
%%writefile tensor_parallel_mlp_dtensor.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
from torch.distributed.device_mesh import init_device_mesh
from torch.distributed._tensor import DTensor, DeviceMesh, Replicate, Shard
import torch.distributed.tensor.parallel as tp


class LlamaMLP(nn.Module):  # same module, but with smaller dims for quick prototyping
    def __init__(self, hidden_size: int = 1024, intermediate_size: int = 4096):
        super().__init__()
        self.gate_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.up_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.down_proj = nn.Linear(intermediate_size, hidden_size, bias=False)

    def forward(self, input):
        return self.down_proj(F.silu(self.gate_proj(input)) * self.up_proj(input))


if __name__ == "__main__":
    dist.init_process_group("gloo")  # use nccl for cuda devices
    torch.manual_seed(1337)          # init weights equally on all ranks
    rank, world_size = dist.get_rank(), dist.get_world_size()

    # Initialize device mesh for tensor parallelism
    device_mesh = init_device_mesh(device_type="cpu", mesh_shape=(world_size,))  # use "cuda" for GPU

    # Create reference module for comparison
    ref_module = nn.Sequential(nn.RMSNorm(1024), LlamaMLP())

    input = torch.randn(1, 1024, requires_grad=True)
    ref_output = ref_module(input)
    ref_output.sum().backward()
    ref_input_grad = input.grad.clone()

    # Create tensor parallel module (we wrap ref_module instead of copying)
    tp_module = tp.parallelize_module(
        ref_module,
        device_mesh,
        parallelize_plan={  # define parallelism type for each module
            # up_proj and gate_proj are column-wise parallel (sharded across outputs);
            "1.up_proj": tp.ColwiseParallel(),
            "1.gate_proj": tp.ColwiseParallel(),
            # down_proj is row-wise parallel (sharded across input dim)
            "1.down_proj": tp.RowwiseParallel(),
          },  # note: RMSNorm is simply replicated across all devices - hence, we skip it
    )
    if rank == 0:  # Note: no need to copy weight chunks manually: DTensor handles parameter sharding for us
      for name, param in tp_module.named_parameters():
        print(f"{name=},\ttype={type(param.data)}\tglobal shape={param.shape},\tlocal shape={param._local_tensor.shape if hasattr(param, '_local_tensor') else param.shape}")

    dist.barrier()  # Test forward and backward pass with Tensor Parallelism
    tp_input = input.detach().requires_grad_(True)
    tp_output = tp_module(tp_input)
    tp_output.sum().backward()
    tp_output = tp_output.trigger_wait()  # convert from AsyncCollectiveTensor to regular torch tensor
    if rank == 0:
        print(f"\nReference outputs ({rank=}):", ref_output.data, flush=True)
        print(f"TParallel outputs ({rank=}):", tp_output.data, flush=True)
        print(f"\nReference input grad ({rank=}):", ref_input_grad, flush=True)
        print(f"TParallel input grad ({rank=}):", tp_input.grad, flush=True)
    dist.barrier()
    assert torch.allclose(tp_output, ref_output, atol=1e-6), f"output mismatch on {rank=}"
    assert torch.allclose(tp_input.grad, ref_input_grad, atol=1e-6), f"input_grad mismatch on {rank=}"
    print(end=f"Tests passed ({rank=})\n", flush=True); dist.barrier()

# fun fact: 90% of the code above was generated by grok-3 for prompt "Please rewrite the following code using torch.distributed.tensor ```python <paste MLP code here>```"
# the remaining 10% are nasty bugfixes that took 99% of assignment preparation time. Do not trust the shogoths yet :)

Writing tensor_parallel_mlp_dtensor.py


In [17]:
!OMP_NUM_THREADS=1 torchrun --nproc_per_node 2 tensor_parallel_mlp_dtensor.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[Gloo] Rank 1 is connected to 1 peer ranks. Expected number of connected peer ranks is : 1
[Gloo] Rank 0 is connected to 1 peer ranks. Expected number of connected peer ranks is : 1
[Gloo] Rank [Gloo] Rank 10 is connected to  is connected to 11 peer ranks.  peer ranks. Expected number of connected peer ranks is : Expected number of connected peer ranks is : 11

name='0.weight',	type=<class 'torch.Tensor'>	global shape=torch.Size([1024]),	local shape=torch.Size([1024])
name='1.gate_proj.weight',	type=<class 'torch.distributed.tensor.DTensor'>	global shape=torch.Size([4096, 1024]),	local shape=torch.Size([2048, 1024])
name='1.up_proj.weight',	type=<class 'torch.distributed.tensor.DTensor'>	global shape=torch.Size([4096, 1024]),	local shape=torch.Size([2048, 1024])
name='1.down_proj.weight',	type=<class 'torch.distributed.tensor.DTensor'>	global shape=torch.Size([1024, 4096]),	local shape=torch.Size([1024, 2048])

Reference outputs (rank=0): tensor([[ 0.0102,  0.0432, -0.0467,  ...,  0.07

In [20]:
%%writefile tensor_parallel_attn_dtensor.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist

from transformers.models.llama.modeling_llama import LlamaConfig, LlamaAttention, LlamaRotaryEmbedding
from torch import nn
import torch
from typing import Dict, Callable, Optional, Tuple
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
from torch.distributed.device_mesh import init_device_mesh
from torch.distributed._tensor import DTensor, DeviceMesh, Replicate, Shard
import torch.distributed.tensor.parallel as tp


MODEL_NAME = "unsloth/Llama-3.2-1B"  # for testing (but not grading!), you may want to use Maykeye/TinyLLama-v0


def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    This is the equivalent of torch.repeat_interleave(x, dim=1, repeats=n_rep). The hidden states go from (batch,
    num_key_value_heads, seqlen, head_dim) to (batch, num_attention_heads, seqlen, head_dim)
    """
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)


def sdpa_attention_forward(
    module: torch.nn.Module,
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    attention_mask: Optional[torch.Tensor],
    dropout: float = 0.0,
    scaling: Optional[float] = None,
    is_causal: Optional[bool] = None,
    **kwargs,
) -> Tuple[torch.Tensor, None]:
    if hasattr(module, "num_key_value_groups"):
        key = repeat_kv(key, module.num_key_value_groups)
        value = repeat_kv(value, module.num_key_value_groups)

    causal_mask = attention_mask
    if attention_mask is not None:
        causal_mask = causal_mask[:, :, :, : key.shape[-2]]

    query = query.contiguous()
    key = key.contiguous()
    value = value.contiguous()

    if is_causal is None:
        is_causal = causal_mask is None and query.shape[2] > 1

    if torch.jit.is_tracing() and isinstance(is_causal, torch.Tensor):
        is_causal = is_causal.item()

    # print('query', query.shape)
    # print('key', key.shape)
    # print('value', value.shape)
    
    attn_output = torch.nn.functional.scaled_dot_product_attention(
        query,
        key,
        value,
        attn_mask=causal_mask,
        dropout_p=dropout,
        scale=scaling,
        is_causal=is_causal,
    )
    attn_output = attn_output.transpose(1, 2).contiguous()

    return attn_output, None


def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(q, k, cos, sin, position_ids=None, unsqueeze_dim=1):
    cos = cos.unsqueeze(unsqueeze_dim)
    sin = sin.unsqueeze(unsqueeze_dim)
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed


class MyLlamaAttention(nn.Module):
    """Multi-headed attention from 'Attention Is All You Need' paper"""

    def __init__(self, config: LlamaConfig, layer_idx: int):
        super().__init__()
        self.config = config
        self.layer_idx = layer_idx
        self.head_dim = getattr(config, "head_dim", config.hidden_size // config.num_attention_heads)
        self.num_key_value_groups = config.num_attention_heads // config.num_key_value_heads
        self.scaling = self.head_dim**-0.5
        self.attention_dropout = config.attention_dropout
        self.is_causal = True

        self.q_proj = nn.Linear(
            config.hidden_size, config.num_attention_heads * self.head_dim, bias=config.attention_bias
        )
        self.k_proj = nn.Linear(
            config.hidden_size, config.num_key_value_heads * self.head_dim, bias=config.attention_bias
        )
        self.v_proj = nn.Linear(
            config.hidden_size, config.num_key_value_heads * self.head_dim, bias=config.attention_bias
        )
        self.o_proj = nn.Linear(
            config.num_attention_heads * self.head_dim, config.hidden_size, bias=config.attention_bias
        )

    def forward(
        self,
        hidden_states: torch.Tensor,
        position_embeddings: Tuple[torch.Tensor, torch.Tensor],
        attention_mask: Optional[torch.Tensor],
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        input_shape = hidden_states.shape[:-1]
        hidden_shape = (*input_shape, -1, self.head_dim)

        query_states = self.q_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        key_states = self.k_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        value_states = self.v_proj(hidden_states).view(hidden_shape).transpose(1, 2)

        cos, sin = position_embeddings
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)

        # print('query_states', query_states.shape)
        # print('key_states', key_states.shape)
        # print('value_states', value_states.shape)
        
        attn_output, attn_weights = sdpa_attention_forward(
            self,
            query_states,
            key_states,
            value_states,
            attention_mask,
            dropout=0.0 if not self.training else self.attention_dropout,
            scaling=self.scaling,
        )

        # print('attn_output', attn_output.shape)

        attn_output = attn_output.reshape(*input_shape, -1).contiguous()
        attn_output = self.o_proj(attn_output)
        return attn_output


class ComputeWithAllReduce(torch.autograd.Function):
    @staticmethod  # fun fact: torch.distributed.nn has differentiable all_reduce!
    def forward(ctx, tp_shard: nn.Module, input: torch.Tensor, **kwargs):
        input = input.detach().requires_grad_(input.requires_grad)
        ctx.save_for_backward(input)
        ctx._tp_shard = tp_shard
        output = tp_shard(input, **kwargs)
        dist.all_reduce(output)
        return output
    @staticmethod
    def backward(ctx, grad_output: torch.Tensor):
        with torch.enable_grad():
          output = ctx._tp_shard(ctx.saved_tensors[0])
          output.backward(grad_output)
        dist.all_reduce(ctx.saved_tensors[0].grad)
        return None, ctx.saved_tensors[0].grad


class ReduceWrapper(nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module

    def forward(self, input, **kwargs):
        fn = lambda x: self.module(x, **kwargs)
        return ComputeWithAllReduce.apply(fn, input)



if __name__ == "__main__":
    dist.init_process_group("gloo")   # use nccl for cuda devices
    torch.manual_seed(1337)           # init weights equally on all ranks
    rank, world_size = dist.get_rank(), dist.get_world_size()

    device_mesh = init_device_mesh(device_type="cpu", mesh_shape=(world_size,))

    config = LlamaConfig.from_pretrained(MODEL_NAME)
    ref_module = MyLlamaAttention(config, layer_idx=5).cpu()
    rotary_emb = LlamaRotaryEmbedding(config).cpu()

    input = torch.randn(1, 128, 2048, requires_grad=True, device='cpu')
    position_embeddings = rotary_emb(input, position_ids=torch.arange(128)[None].cpu())
    extra_inputs = dict(attention_mask=None, position_embeddings=position_embeddings)  # <-- OPTIONAL: either design additional inputs here, as in the reference above

    ref_output = ref_module(input, **extra_inputs).cpu()
    ref_output.sum().backward()
    ref_input_grad = input.grad.clone()

    tp_module = tp.parallelize_module(
        ref_module,
        device_mesh,
        parallelize_plan={
            "q_proj": tp.ColwiseParallel(),
            "k_proj": tp.ColwiseParallel(),
            "v_proj": tp.ColwiseParallel(),
            "o_proj": tp.RowwiseParallel(),
        }
    )

    if rank == 0:
        for name, param in tp_module.named_parameters():
            print(f"{name=},\ttype={type(param.data)}\tglobal shape={param.shape},\tlocal shape={param._local_tensor.shape if hasattr(param, '_local_tensor') else param.shape}")
    
    dist.barrier()
    tp_input = input.detach().requires_grad_(True)
    tp_output = tp_module(tp_input, **extra_inputs)
    tp_output.sum().backward()
    tp_output = tp_output.trigger_wait()
    if rank == 0:
        print(f"\nReference outputs ({rank=}):", ref_output.data, flush=True)
        print(f"TParallel outputs ({rank=}):", tp_output.data, flush=True)
        print(f"\nReference input grad ({rank=}):", ref_input_grad, flush=True)
        print(f"TParallel input grad ({rank=}):", tp_input.grad, flush=True)
    dist.barrier()
    assert torch.allclose(tp_output, ref_output, atol=1e-6), f"output mismatch on {rank=}"
    assert torch.allclose(tp_input.grad, ref_input_grad, atol=1e-6), f"input_grad mismatch on {rank=}"
    print(end=f"Tests passed ({rank=})\n", flush=True); dist.barrier()

Overwriting tensor_parallel_attn_dtensor.py


In [21]:
!OMP_NUM_THREADS=1 torchrun --nproc_per_node 2 tensor_parallel_attn_dtensor.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[Gloo] Rank 0 is connected to 1 peer ranks. Expected number of connected peer ranks is : 1[Gloo] Rank 
1 is connected to 1 peer ranks. Expected number of connected peer ranks is : 1
[Gloo] Rank 0[Gloo] Rank  is connected to 11 is connected to  peer ranks. 1Expected number of connected peer ranks is :  peer ranks. 1Expected number of connected peer ranks is : 
1
name='q_proj.weight',	type=<class 'torch.distributed.tensor.DTensor'>	global shape=torch.Size([2048, 2048]),	local shape=torch.Size([1024, 2048])
name='k_proj.weight',	type=<class 'torch.distributed.tensor.DTensor'>	global shape=torch.Size([512, 2048]),	local shape=torch.Size([256, 2048])
name='v_proj.weight',	type=<class 'torch.distributed.tensor.DTensor'>	global shape=torch.Size([512, 2048]),	local shape=torch.Size([256, 2048])
name='o_proj.weight',	type=<class 'torch.distributed.tensor.DTensor'>	global shape=torch.Size([2048, 2048]),	local shape=torch.Size([2048, 1024])

Reference outputs (rank=0): tensor([[[-0.1457, -0.3905,

In [52]:
%%writefile tensor_parallel_llama_dtensor.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist

from transformers.models.llama.modeling_llama import LlamaConfig, LlamaAttention, LlamaRotaryEmbedding
from torch import nn
import torch
from typing import Dict, Callable, Optional, Tuple
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
from torch.distributed.device_mesh import init_device_mesh
from torch.distributed._tensor import DTensor, DeviceMesh, Replicate, Shard
import torch.distributed.tensor.parallel as tp
import transformers


MODEL_NAME = "unsloth/Llama-3.2-1B"  # for testing (but not grading!), you may want to use Maykeye/TinyLLama-v0


def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    This is the equivalent of torch.repeat_interleave(x, dim=1, repeats=n_rep). The hidden states go from (batch,
    num_key_value_heads, seqlen, head_dim) to (batch, num_attention_heads, seqlen, head_dim)
    """
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)


def sdpa_attention_forward(
    module: torch.nn.Module,
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    attention_mask: Optional[torch.Tensor],
    dropout: float = 0.0,
    scaling: Optional[float] = None,
    is_causal: Optional[bool] = None,
    **kwargs,
) -> Tuple[torch.Tensor, None]:
    if hasattr(module, "num_key_value_groups"):
        key = repeat_kv(key, module.num_key_value_groups)
        value = repeat_kv(value, module.num_key_value_groups)

    causal_mask = attention_mask
    if attention_mask is not None:
        causal_mask = causal_mask[:, :, :, : key.shape[-2]]

    query = query.contiguous()
    key = key.contiguous()
    value = value.contiguous()

    if is_causal is None:
        is_causal = causal_mask is None and query.shape[2] > 1

    if torch.jit.is_tracing() and isinstance(is_causal, torch.Tensor):
        is_causal = is_causal.item()

    # print('query', query.shape)
    # print('key', key.shape)
    # print('value', value.shape)
    
    attn_output = torch.nn.functional.scaled_dot_product_attention(
        query,
        key,
        value,
        attn_mask=causal_mask,
        dropout_p=dropout,
        scale=scaling,
        is_causal=is_causal,
    )
    attn_output = attn_output.transpose(1, 2).contiguous()

    return attn_output, None


def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(q, k, cos, sin, position_ids=None, unsqueeze_dim=1):
    cos = cos.unsqueeze(unsqueeze_dim)
    sin = sin.unsqueeze(unsqueeze_dim)
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed


class MyLlamaAttention(nn.Module):
    """Multi-headed attention from 'Attention Is All You Need' paper"""

    def __init__(self, config: LlamaConfig, layer_idx: int):
        super().__init__()
        self.config = config
        self.layer_idx = layer_idx
        self.head_dim = getattr(config, "head_dim", config.hidden_size // config.num_attention_heads)
        self.num_key_value_groups = config.num_attention_heads // config.num_key_value_heads
        self.scaling = self.head_dim**-0.5
        self.attention_dropout = config.attention_dropout
        self.is_causal = True

        self.q_proj = nn.Linear(
            config.hidden_size, config.num_attention_heads * self.head_dim, bias=config.attention_bias
        )
        self.k_proj = nn.Linear(
            config.hidden_size, config.num_key_value_heads * self.head_dim, bias=config.attention_bias
        )
        self.v_proj = nn.Linear(
            config.hidden_size, config.num_key_value_heads * self.head_dim, bias=config.attention_bias
        )
        self.o_proj = nn.Linear(
            config.num_attention_heads * self.head_dim, config.hidden_size, bias=config.attention_bias
        )

    def forward(
        self,
        hidden_states: torch.Tensor,
        position_embeddings: Tuple[torch.Tensor, torch.Tensor],
        attention_mask: Optional[torch.Tensor],
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        input_shape = hidden_states.shape[:-1]
        hidden_shape = (*input_shape, -1, self.head_dim)

        query_states = self.q_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        key_states = self.k_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        value_states = self.v_proj(hidden_states).view(hidden_shape).transpose(1, 2)

        cos, sin = position_embeddings
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)

        # print('query_states', query_states.shape)
        # print('key_states', key_states.shape)
        # print('value_states', value_states.shape)
        
        attn_output, attn_weights = sdpa_attention_forward(
            self,
            query_states,
            key_states,
            value_states,
            attention_mask,
            dropout=0.0 if not self.training else self.attention_dropout,
            scaling=self.scaling,
        )

        # print('attn_output', attn_output.shape)

        attn_output = attn_output.reshape(*input_shape, -1).contiguous()
        attn_output = self.o_proj(attn_output)
        return attn_output


class ComputeWithAllReduce(torch.autograd.Function):
    @staticmethod  # fun fact: torch.distributed.nn has differentiable all_reduce!
    def forward(ctx, tp_shard: nn.Module, input: torch.Tensor, **kwargs):
        input = input.detach().requires_grad_(input.requires_grad)
        ctx.save_for_backward(input)
        ctx._tp_shard = tp_shard
        output = tp_shard(input, **kwargs)
        dist.all_reduce(output)
        return output
    @staticmethod
    def backward(ctx, grad_output: torch.Tensor):
        with torch.enable_grad():
          output = ctx._tp_shard(ctx.saved_tensors[0])
          output.backward(grad_output)
        dist.all_reduce(ctx.saved_tensors[0].grad)
        return None, ctx.saved_tensors[0].grad


class ReduceWrapper(nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module

    def forward(self, input, **kwargs):
        fn = lambda x: self.module(x, **kwargs)
        return ComputeWithAllReduce.apply(fn, input)



if __name__ == "__main__":
    dist.init_process_group("nccl")   # use nccl for cuda devices
    torch.manual_seed(1337)           # init weights equally on all ranks
    rank, world_size = dist.get_rank(), dist.get_world_size()
    torch.cuda.set_device(rank)

    device_mesh = init_device_mesh(device_type="cuda", mesh_shape=(world_size,))

    MODEL_NAME = "unsloth/Llama-3.2-1B"  # for testing (but not grading!), you may want to use Maykeye/TinyLLama-v0
    config = LlamaConfig.from_pretrained(MODEL_NAME)
    tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
    ref_model = transformers.LlamaForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float32).cuda()  # <-- you are allowed to switch to bf16
    prompt = "A quick brown fox"
    input_ids = tokenizer(prompt, return_tensors='pt')["input_ids"].cuda()
    input = ref_model.model.embed_tokens(input_ids).detach().requires_grad_(True)
    ref_output = ref_model(inputs_embeds=input).logits
    ref_output.mean().backward()
    ref_input_grad = input.grad.clone()

    parallelize_plan = {}

    for i in range(len(ref_model.model.layers)):
        parallelize_plan[f"model.layers.{i}.self_attn.q_proj"] = tp.ColwiseParallel()
        parallelize_plan[f"model.layers.{i}.self_attn.k_proj"] = tp.ColwiseParallel()
        parallelize_plan[f"model.layers.{i}.self_attn.v_proj"] = tp.ColwiseParallel()
        parallelize_plan[f"model.layers.{i}.self_attn.o_proj"] = tp.RowwiseParallel()
        parallelize_plan[f"model.layers.{i}.mlp.gate_proj"] = tp.ColwiseParallel()
        parallelize_plan[f"model.layers.{i}.mlp.down_proj"] = tp.RowwiseParallel()
        parallelize_plan[f"model.layers.{i}.mlp.up_proj"] = tp.ColwiseParallel()
        
    
    tp_module = tp.parallelize_module(
        ref_model,
        device_mesh,
        parallelize_plan=parallelize_plan
    )

    # if rank == 0:
    #     for name, param in tp_module.named_parameters():
    #         print(f"{name=},\ttype={type(param.data)}\tglobal shape={param.shape},\tlocal shape={param._local_tensor.shape if hasattr(param, '_local_tensor') else param.shape}")
    
    dist.barrier()
    tp_input = input.detach().requires_grad_(True).cuda()
    tp_output = tp_module(inputs_embeds=tp_input).logits
    tp_output.mean().backward()
    if rank == 0:
        print(f"\nReference outputs ({rank=}):", ref_output.data, flush=True)
        print(f"TParallel outputs ({rank=}):", tp_output.data, flush=True)
        print(f"\nReference input grad ({rank=}):", ref_input_grad, flush=True)
        print(f"TParallel input grad ({rank=}):", tp_input.grad, flush=True)
    dist.barrier()
    assert torch.allclose(tp_output, ref_output, atol=1e-3), f"output mismatch on {rank=}"
    assert torch.allclose(tp_input.grad, ref_input_grad, atol=1e-3), f"input_grad mismatch on {rank=}"
    print(end=f"Tests passed ({rank=})\n", flush=True); dist.barrier()

    dist.barrier()
    text = "A quick brown"
    input_ids = tokenizer(text, return_tensors='pt')["input_ids"].cuda()
    for i in range(10):
        with torch.no_grad():
            new_token = tp_module(input_ids).logits[0, -1].argmax(-1)
            input_ids = torch.cat([input_ids, new_token.view(1, 1)], dim=1)
    final_text = tokenizer.decode(input_ids[0])
    print(end=f"\n\nText generation ({rank=}):\n{final_text}\n", flush=True)
    dist.barrier()

Overwriting tensor_parallel_llama_dtensor.py


In [54]:
%%time
!OMP_NUM_THREADS=1 torchrun --nproc_per_node 2 tensor_parallel_llama_dtensor.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/venv/main/lib/python3.10/site-packages/torch/distributed/c10d_logger.py:83: UserWarning: barrier(): using the device under current context. You can specify `device_id` in `init_process_group` to mute this warning.
  return func(*args, **kwargs)

Reference outputs (rank=0): tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [ 8.7207,  7.6674,  7.7454,  ..., -1.3887, -1.3885, -1.3885],
         [ 9.7538,  7.3480,  5.6021,  ..., -0.3843, -0.3842, -0.3834],
         [10.0018,  7.8891,  5.5960,  ..., -2.3212, -2.3206, -2.3199],
         [12.8186, 10.7330,  8.5906,  ..., -1.0288, -1.0301, -1.0296]]],
       device='cuda:0')
TParallel outputs (rank=0): tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [ 8.7207,  7.6674,  7.7454,  ..., -1.3887, -1.3885, -1.3885],
         [ 9.7538,  7.3480,  5.6021,  ..., -0.3843, -0.3842, -0.3834],
         [10.0018,  7.8891,  5.5960,  ..., -2.3212, -2.3206, -2.3199],
         [12.8186, 10.7330,  8.5

In [ ]:
%%writefile tensor_parallel_llama_cuda.py
import torch
import transformers
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist

from transformers.models.llama.modeling_llama import LlamaConfig, LlamaAttention, LlamaRotaryEmbedding
from torch import nn
import torch
from typing import Dict, Callable, Optional, Tuple
from copy import deepcopy


MODEL_NAME = "unsloth/Llama-3.2-1B"  # for testing (but not grading!), you may want to use Maykeye/TinyLLama-v0


def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    This is the equivalent of torch.repeat_interleave(x, dim=1, repeats=n_rep). The hidden states go from (batch,
    num_key_value_heads, seqlen, head_dim) to (batch, num_attention_heads, seqlen, head_dim)
    """
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)


def sdpa_attention_forward(
    module: torch.nn.Module,
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    attention_mask: Optional[torch.Tensor],
    dropout: float = 0.0,
    scaling: Optional[float] = None,
    is_causal: Optional[bool] = None,
    **kwargs,
) -> Tuple[torch.Tensor, None]:
    if hasattr(module, "num_key_value_groups"):
        key = repeat_kv(key, module.num_key_value_groups)
        value = repeat_kv(value, module.num_key_value_groups)

    causal_mask = attention_mask
    if attention_mask is not None:
        causal_mask = causal_mask[:, :, :, : key.shape[-2]]

    query = query.contiguous()
    key = key.contiguous()
    value = value.contiguous()

    if is_causal is None:
        is_causal = causal_mask is None and query.shape[2] > 1

    if torch.jit.is_tracing() and isinstance(is_causal, torch.Tensor):
        is_causal = is_causal.item()

    # print('query', query.shape)
    # print('key', key.shape)
    # print('value', value.shape)
    
    attn_output = torch.nn.functional.scaled_dot_product_attention(
        query,
        key,
        value,
        attn_mask=causal_mask,
        dropout_p=dropout,
        scale=scaling,
        is_causal=is_causal,
    )
    attn_output = attn_output.transpose(1, 2).contiguous()

    return attn_output, None


def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(q, k, cos, sin, position_ids=None, unsqueeze_dim=1):
    cos = cos.unsqueeze(unsqueeze_dim)
    sin = sin.unsqueeze(unsqueeze_dim)
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed


class MyLlamaAttention(nn.Module):
    """Multi-headed attention from 'Attention Is All You Need' paper"""

    def __init__(self, config: LlamaConfig, layer_idx: int):
        super().__init__()
        self.config = config
        self.layer_idx = layer_idx
        self.head_dim = getattr(config, "head_dim", config.hidden_size // config.num_attention_heads)
        self.num_key_value_groups = config.num_attention_heads // config.num_key_value_heads
        self.scaling = self.head_dim**-0.5
        self.attention_dropout = config.attention_dropout
        self.is_causal = True

        self.q_proj = nn.Linear(
            config.hidden_size, config.num_attention_heads * self.head_dim, bias=config.attention_bias
        )
        self.k_proj = nn.Linear(
            config.hidden_size, config.num_key_value_heads * self.head_dim, bias=config.attention_bias
        )
        self.v_proj = nn.Linear(
            config.hidden_size, config.num_key_value_heads * self.head_dim, bias=config.attention_bias
        )
        self.o_proj = nn.Linear(
            config.num_attention_heads * self.head_dim, config.hidden_size, bias=config.attention_bias
        )

    def forward(
        self,
        hidden_states: torch.Tensor,
        position_embeddings: Tuple[torch.Tensor, torch.Tensor],
        attention_mask: Optional[torch.Tensor], **kwargs,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        input_shape = hidden_states.shape[:-1]
        hidden_shape = (*input_shape, -1, self.head_dim)

        query_states = self.q_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        key_states = self.k_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        value_states = self.v_proj(hidden_states).view(hidden_shape).transpose(1, 2)

        cos, sin = position_embeddings
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)

        # print('query_states', query_states.shape)
        # print('key_states', key_states.shape)
        # print('value_states', value_states.shape)
        
        attn_output, attn_weights = sdpa_attention_forward(
            self,
            query_states,
            key_states,
            value_states,
            attention_mask,
            dropout=0.0 if not self.training else self.attention_dropout,
            scaling=self.scaling,
        )

        # print('attn_output', attn_output.shape)

        attn_output = attn_output.reshape(*input_shape, -1).contiguous()
        attn_output = self.o_proj(attn_output)
        return attn_output


class ComputeWithAllReduce(torch.autograd.Function):
    @staticmethod  # fun fact: torch.distributed.nn has differentiable all_reduce!
    def forward(ctx, tp_shard: nn.Module, input: torch.Tensor, **kwargs):
        input = input.detach().requires_grad_(input.requires_grad)
        ctx.save_for_backward(input)
        ctx._tp_shard = tp_shard
        output = tp_shard(input, **kwargs)
        dist.all_reduce(output)
        return output
    @staticmethod
    def backward(ctx, grad_output: torch.Tensor):
        with torch.enable_grad():
          output = ctx._tp_shard(ctx.saved_tensors[0])
          output.backward(grad_output)
        dist.all_reduce(ctx.saved_tensors[0].grad)
        return None, ctx.saved_tensors[0].grad


class ReduceWrapper(nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module

    def forward(self, hidden_states=None, **kwargs):
        fn = lambda x: self.module(x, **kwargs)
        output = ComputeWithAllReduce.apply(fn, hidden_states)
        return output, None


class LlamaMLP(nn.Module):  #  based on llama 3.1 8B configuration
    def __init__(self, hidden_size: int = 4096, intermediate_size: int = 14336):
        super().__init__()
        self.gate_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.up_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.down_proj = nn.Linear(intermediate_size, hidden_size, bias=False)

    def forward(self, input):
        return self.down_proj(F.silu(self.gate_proj(input)) * self.up_proj(input))


class ComputeWithAllReduce(torch.autograd.Function):
    @staticmethod  # fun fact: torch.distributed.nn has differentiable all_reduce!
    def forward(ctx, tp_shard: nn.Module, input: torch.Tensor):
        input = input.detach().requires_grad_(input.requires_grad)
        ctx.save_for_backward(input)
        ctx._tp_shard = tp_shard
        output = tp_shard(input)
        dist.all_reduce(output)
        return output
    @staticmethod
    def backward(ctx, grad_output: torch.Tensor):
        with torch.enable_grad():
          output = ctx._tp_shard(ctx.saved_tensors[0])
          output.backward(grad_output)
        dist.all_reduce(ctx.saved_tensors[0].grad)
        return None, ctx.saved_tensors[0].grad


class AllReduceModule(nn.Sequential):
    def forward(self, input: torch.Tensor):
        return ComputeWithAllReduce.apply(super().forward, input)


if __name__ == "__main__":
    dist.init_process_group("nccl")   # use nccl for cuda devices
    torch.manual_seed(1337)           # init weights equally on all ranks
    rank, world_size = dist.get_rank(), dist.get_world_size()
    torch.cuda.set_device(rank)

    for active_rank in range(world_size):
        dist.barrier()
        if rank != active_rank: continue
        
        MODEL_NAME = "unsloth/Llama-3.2-1B"  # for testing (but not grading!), you may want to use Maykeye/TinyLLama-v0
        config = LlamaConfig.from_pretrained(MODEL_NAME)
        tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
        ref_model = transformers.LlamaForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float32).cuda()  # <-- you are allowed to switch to bf16
        prompt = "A quick brown fox"
        input_ids = tokenizer(prompt, return_tensors='pt')["input_ids"].cuda()
        input = ref_model.model.embed_tokens(input_ids).detach().requires_grad_(True)
        ref_output = ref_model(inputs_embeds=input).logits
        ref_output.mean().backward()
        ref_input_grad = input.grad.clone()
        
        tp_model = deepcopy(ref_model)
        
        with torch.no_grad():
            for i in range(len(tp_model.model.layers)):
                smaller_config = deepcopy(config)
                smaller_config.num_attention_heads = config.num_attention_heads // world_size
                smaller_config.num_key_value_heads = config.num_key_value_heads // world_size
                layer = ReduceWrapper(MyLlamaAttention(smaller_config, layer_idx=i)).cuda()
                local_units = config.head_dim * smaller_config.num_key_value_heads
                unit_slice = slice(local_units * rank, local_units * (rank + 1))
                layer.module.k_proj.weight.data = ref_model.model.layers[i].self_attn.k_proj.weight.data[unit_slice]
                layer.module.v_proj.weight.data = ref_model.model.layers[i].self_attn.v_proj.weight.data[unit_slice]
                local_units = config.head_dim * smaller_config.num_attention_heads
                unit_slice = slice(local_units * rank, local_units * (rank + 1))
                layer.module.o_proj.weight.data[...] = ref_model.model.layers[i].self_attn.o_proj.weight.data[:, unit_slice]
                layer.module.q_proj.weight.data[...] = ref_model.model.layers[i].self_attn.q_proj.weight.data[unit_slice]
                tp_model.model.layers[i].self_attn = layer

                intermediate_size = ref_model.model.layers[i].mlp.down_proj.in_features
                local_units = intermediate_size // world_size
                
                mlp_layer = AllReduceModule(LlamaMLP(hidden_size=config.hidden_size, intermediate_size=local_units)).cuda()
                unit_slice = slice(local_units * rank, local_units * (rank + 1))
                
                mlp_layer[0].down_proj.weight.data[...] = ref_model.model.layers[i].mlp.down_proj.weight.data[:, unit_slice]
                mlp_layer[0].up_proj.weight.data[...] = ref_model.model.layers[i].mlp.up_proj.weight.data[unit_slice]
                mlp_layer[0].gate_proj.weight.data[...] = ref_model.model.layers[i].mlp.gate_proj.weight.data[unit_slice]
                tp_model.model.layers[i].mlp = mlp_layer
    
    dist.barrier()
    tp_input = input.detach().requires_grad_(True).cuda()
    tp_output = tp_model(inputs_embeds=tp_input).logits
    if rank == 0:
        print(f"\nReference outputs ({rank=}):", ref_output.data, flush=True)
    for i in range(world_size):
        dist.barrier()
        if i != rank: continue
        print(f"TParallel outputs ({rank=}):", tp_output.data, flush=True)
        assert torch.allclose(tp_output, ref_output, atol=1e-3), f"output mismatch on {rank=}"
    
    dist.barrier()  # test 2: backward w.r.t. inputs
    assert tp_input.grad is None
    tp_output.mean().backward()
    if rank == 0:
        print(f"\nReference input grad ({rank=}):", ref_input_grad, flush=True)
    for i in range(world_size):
        dist.barrier()
        if i != rank: continue
        print(f"TParallel input grad ({rank=}):", tp_input.grad.data, flush=True)
        assert torch.allclose(tp_input.grad, ref_input_grad, atol=1e-3), f"input_grad mismatch on {rank=}"
    
    dist.barrier()
    text = "A quick brown"
    input_ids = tokenizer(text, return_tensors='pt')["input_ids"].cuda()
    for i in range(10):
        with torch.no_grad():
            new_token = tp_model(input_ids).logits[0, -1].argmax(-1)
            input_ids = torch.cat([input_ids, new_token.view(1, 1)], dim=1)
        # print(end=tokenizer.decode(new_token), flush=True)
    final_text = tokenizer.decode(input_ids[0])
    print(end=f"\n\nText generation ({rank=}):\n{final_text}\n", flush=True)
    dist.barrier()

Writing tensor_parallel_llama_cuda.py


In [56]:
%%time
!OMP_NUM_THREADS=1 torchrun --nproc_per_node 2 tensor_parallel_llama_cuda.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/venv/main/lib/python3.10/site-packages/torch/distributed/c10d_logger.py:83: UserWarning: barrier(): using the device under current context. You can specify `device_id` in `init_process_group` to mute this warning.
  return func(*args, **kwargs)
[rank0]:[W223 08:34:04.227183287 ProcessGroupNCCL.cpp:5138] Guessing device ID based on global rank. This can cause a hang if rank to GPU mapping is heterogeneous. You can specify device_id in init_process_group()

Reference outputs (rank=0): tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [ 8.7207,  7.6674,  7.7454,  ..., -1.3887, -1.3885, -1.3885],
         [ 9.7538,  7.3480,  5.6021,  ..., -0.3843, -0.3842, -0.3834],
         [10.0018,  7.8891,  5.5960,  ..., -2.3212, -2.3206, -2.3199],
         [12.8186, 10.7330,  8.5906,  ..., -1.0288, -1.0301, -1.0296]]],
       device='cuda:0')
TParallel outputs (rank=0): tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [ 8.7207,  7.6674,  7.

большая часть времени уходит на то чтобы начать обрабатывать 

```

```

```

```

```

```

```

```

```

```

```

```

```

```

```

```



### Sequence Parallelism with Ulysses


Now let's parallelize the other way - across the sequence dimension. To showcase why this is necessary, our main task will be to parallelize LLM fine-tuning over a very long sequence. The way you do this, of course, is through Sequence Parallelism. You can implement naive [sequence parallelism](https://arxiv.org/abs/2205.05198), similar to [DeepSpeed Ulysses](https://arxiv.org/pdf/2309.14509) (n.b.: not the first work to do this).

![figure-from-paper](https://ar5iv.labs.arxiv.org/html/2309.14509/assets/figs/image3.png)


Here's the short version:
- All weights are replicated between ranks (optionally: FSDP)
- Each rank holds a subset of sequence tokens
- Embeddings, logits, normalizations, MLP all apply independently to token shards
- The multi-head attention is the only layer that gets special treatment
    - First, apply QKV projections to local tokens, as in data-parallel training;
    - Then re-shard so that each rank holds a **subset of heads** across **all tokens**;
    - Compute the attention ''core'' (RoPE and F.scaled_dot_product_attention) for its chunk of heads independently;
    - Re-shard outputs again so that each rank concatenates **all heads**, but only for its **subset of tokens**;
    - Apply the output ("O") projection to your local tokens again.
- This approach *may* be combined with tensor parallelism, but this is an advanced technique that you don't have to implement.


__You have a choice__ between two options on how to implement it: either manually with torch.distributed like in task 2, or using the DTensor route like in task 3. We provide some tips for both tasks.


**Option A. with raw `torch.distirbuted`:**
- Use [`dist.all_to_all`](https://pytorch.org/docs/stable/distributed.html#torch.distributed.all_to_all) to switch between per-token and per-head sharding without materializing the full tensor on any device;
- Wrap the model with [`DistributedDataParallel`](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html) or [`FullyShardedDataParallel`](https://pytorch.org/docs/stable/fsdp.html) so that fine-tuning synchronizes trainable parameters. Note that using FSDP for parameter-efficient fine-tuning can be tricky: we recommend you either wrap **trainable modules** with separate FSDP sub-instances via auto_wrap_policy - or simply use DDP instead of FSDP.

**Option B. with `DTensor`:**
- We recommend you first skim the official [tutorial](https://pytorch.org/tutorials/intermediate/TP_tutorial.html) on applying Tensor Parallelism (sic.) - or browse the [TorchTitan's version](https://github.com/pytorch/torchtitan/blob/82afc842e303e49d1a137fc7ea48291a57f72d5d/torchtitan/models/llama/parallelize_llama.py) of it.
- Note that there is a [`SequenceParallel`](https://pytorch.org/docs/stable/distributed.tensor.parallel.html#torch.distributed.tensor.parallel.SequenceParallel) class in torch.distributed.tensor.parallel` - **however, it does not magick the sequence parallelism for you** - it is only meant for small layers (e.g. normalization). You still need to do the sharding in self-attention!

For the sake of formality, here's an example script you need to parallelize:

In [ ]:
import torch
import transformers
import peft
MODEL_NAME = "unsloth/Llama-3.2-1B"  # for testing (but not grading!), you may want to use Maykeye/TinyLLama-v0
SEQUENCE_LENGTH = 128                # IMPORTANT!!! you need to increase this parameter! Look for the maximum sequence length on one and multiple GPUs

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
model = transformers.LlamaForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.bfloat16).to(device)

for param in model.parameters():
  param.required_grad = False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model = peft.get_peft_model(model, peft.PromptTuningConfig(task_type=peft.TaskType.CAUSAL_LM, num_virtual_tokens=32))
assert any(param.requires_grad for param in model.parameters()), "No trainable parameters - did you enable PEFT?"

!wget -q https://www.gutenberg.org/cache/epub/4300/pg4300.txt -O ulysses.txt  # ... or use any other text of your choosing
input_ids = tokenizer(open("ulysses.txt").read(), return_tensors='pt')['input_ids']
print(f"Cropping {input_ids.shape[1]=} to {SEQUENCE_LENGTH} tokens")
input_ids, labels = input_ids[:, :SEQUENCE_LENGTH], input_ids[:, 1:SEQUENCE_LENGTH + 1]

trainable_parameters = {p for p in model.parameters() if p.requires_grad}
print(f"Parameters: {sum(map(torch.Tensor.numel, trainable_parameters))} trainable / {sum(map(torch.Tensor.numel, model.parameters()))} total")
opt = torch.optim.Adam(trainable_parameters)
for i in range(10):
  loss = model(input_ids=input_ids.to(device), labels=labels.to(device)).loss
  opt.zero_grad()
  loss.backward()
  opt.step()
  print(f"{i=}\t{loss.item()=}")

# pro tip: delete the model or restart session to free RAM for the TP experiments

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (397368 > 131072). Running this sequence through the model will result in indexing errors


Cropping input_ids.shape[1]=397368 to 128 tokens
Parameters: 65536 trainable / 1235879936 total
i=0	loss.item()=8.582364082336426
i=1	loss.item()=8.413138389587402
i=2	loss.item()=8.251019477844238
i=3	loss.item()=8.113922119140625
i=4	loss.item()=8.001195907592773
i=5	loss.item()=7.875072002410889
i=6	loss.item()=7.784201145172119
i=7	loss.item()=7.691455364227295
i=8	loss.item()=7.621612548828125
i=9	loss.item()=7.547983169555664


__Task 4 (1 point):__ before you do training, let's first parallelize a single forward pass. Implement sharding with the same interface you used in tasks 2 (or 3 if you use DTensor), but this time, parallelize across the sequence dimension. Note: if you are running out of (V)RAM, load the 1B model in half precision and disable gradients for all weights except the first (few) layers.



In [ ]:
<<A whole lot of your code here>>

In [ ]:
<<... and a dedicated cell to show off that it works>>

__Task 5 (1 point):__ Now use the script above to parallelize the entire training run. You are free to use other fine-tuning methods (e.g. LoRA or even full fine-tuning), as long as you can demonstrate that the loss goes down.

**Make sure you increase SEQUENCE_LENGTH as much as possible!** Even on a single GPU, you should be able to go into thousands, if not tens of thousands of tokens - and report the maximum sequence length with one and with multiple GPUs respectively.

If you don't have access to multiple GPUs, you may optionally submit a version that does training on a single GPU, but computes attention heads sequentially with gradient checkpointing - but if you do, please announce that you are using this option in bold, capital letters, so that the grader will notice it.

In [ ]:
<<A whole lot of your code here>>

In [ ]:
<<... and a dedicated cell to show off that it works>>

```

```

```

```

```

```


### Optional: bonus tasks

There are many routes to further improve the training/inference code. You may (but you don't have to) implement any combination of them for bonus points.

However, please not that the total points for this week's entire assignment (part 1 & 2) are **capped at 14**.

__Bonus task: parallel key-value caching (1 point).__ In tasks 2 and 3, you implement tensor parallelism for attention forward pass and perform inference with re-computation. However, real world inference engines use [KV caching](https://huggingface.co/docs/transformers/main/en/kv_cache) - keeping key and value caches from past tokens and only processing the new token each time.

For this task, you will have to implement this type of parallelism for either torch.distributed or DTensor implementation of attention $-$ simply cache the heads already assigned to each rank. To get the grade, you will need to demonstrate that the model generates a sensible text with any cache (via past_key_values=).

__Bonus task: pipeline parallelism (1-2 points):__ In tasks 1-3, you've implemented symmetric model parallelism, aka Tensor Parallelism. However, there is another way to partition model parameters $-$ assign entire layers to each rank and run them in a pipeline. This can be faster, especially if you are running

For 1 point, check out [torch.distributed.pipelinging](https://pytorch.org/docs/stable/distributed.pipelining.html), [DeepSpeed pipelining](https://deepspeed.readthedocs.io/en/latest/pipeline.html) or [torchgpipe](https://github.com/kakaobrain/torchgpipe) and demonstrate that you can run or fine-tune a model that would not fit into a single GPU (you will need multuple devices for this!).

For 2 points, compare different pipelining schedules in terms of training throughput: use GPipe as a baseline and try ScheduleInterleaved1F1B (or a more advanced pipeline of your choosing).

__Bonus task: better sequence parallelism (2 points).__ In tasks 4 and 5, you implemented basic sequence parallelism. However, there are multiple ways you can improve that technique for further memory savings or better device utilization.

For 1 point, implement combined tensor + sequence parallelism and compare results with naive sequence parallelism.

For 2 points, implement [Ring Attention](https://arxiv.org/abs/2310.01889) *or* integrate computation-communication overlap from [FLUX](https://arxiv.org/abs/2406.06858) and measure the speed and memory trade-offs.